<a href="https://colab.research.google.com/github/sergio2526/prueba_coder_git/blob/analitica/Transformaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear un modelo de machine learning capaz de predecir la empleabildiad de un estudiante, teniendo presente variables de estudio. (Dataset DigitalHouse)

- One Hot Encoding
- Label Encoder
- SimpleImputer

In [ ]:
!pip install scikit-learn --upgrade
#!pip install scipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<hr>

## Descargue el conjunto de datos del desafío en formato .csv

In [ ]:
import pandas as pd
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv


--2022-10-27 16:04:04--  https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654532 (639K) [text/plain]
Saving to: ‘train_dataset_digitalhouse.csv’

train_dataset_digit 100%[===================>] 639.19K  --.-KB/s    in 0.05s   

2022-10-27 16:04:05 (13.5 MB/s) - ‘train_dataset_digitalhouse.csv’ saved [654532/654532]



In [ ]:
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset.tail()

,Unnamed: 0,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
8990,9995,29.0,MASCULINO,ARGENTINA,TERTIARY,DEVELOPMENT,NaN,4.0,4701.6,9.1,86.0
8991,9996,34.0,NaN,ARGENTINA,UNIVERSITARY,ENGINEERING,PROGRAMACION,3.4,4646.2,16.8,95.0
8992,9997,28.0,FEMENINO,ARGENTINA,POST_GRADUATE,ENGINEERING,EJECUTIVO,NaN,3315.1,5.6,95.0
8993,9998,23.0,MASCULINO,MEXICO,TERTIARY,ENGINEERING,PROGRAMACION,3.3,4437.8,0.9,87.0
8994,9999,36.0,MASCULINO,ARGENTINA,UNIVERSITARY,COMMERCIAL,DATA,3.4,4600.8,19.6,88.0


## En el conjunto de datos proporcionado, tenemos las siguientes columnas:

* Unnamed: 0
* EDAD
* GENERO
* RESIDENCIA
* NV_ESTUDIO
* ESTUDIO_PREV
* TRACK_DH
* AVG_DH
* MINUTES_DH
* EXPERIENCIA
* **DIAS_EMP = El valor "target" que se va a predecir**

Podemos verificar fácilmente que faltan valores usando el siguiente código:

```df_training_dataset.info()```

Debe manejar cuidadosamente estos valores faltantes antes de crear un modelo de regresión.

In [ ]:
df_training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8995 entries, 0 to 8994
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8995 non-null   int64  
 1   EDAD          7668 non-null   float64
 2   GENERO        7620 non-null   object 
 3   RESIDENCIA    7638 non-null   object 
 4   NV_ESTUDIO    7623 non-null   object 
 5   ESTUDIO_PREV  7665 non-null   object 
 6   TRACK_DH      7714 non-null   object 
 7   AVG_DH        7651 non-null   float64
 8   MINUTES_DH    7619 non-null   float64
 9   EXPERIENCIA   7618 non-null   float64
 10  DIAS_EMP      8995 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 773.1+ KB


In [ ]:
df_training_dataset['GENERO'].value_counts()

MASCULINO    5230
FEMENINO     2390
Name: GENERO, dtype: int64

In [ ]:
#@title Observando valores nulos
print(df_training_dataset.shape)

(8995, 11)


In [ ]:
100*df_training_dataset.isnull().sum()/df_training_dataset.shape[0]

Unnamed: 0       0.000000
EDAD            14.752640
GENERO          15.286270
RESIDENCIA      15.086159
NV_ESTUDIO      15.252918
ESTUDIO_PREV    14.785992
TRACK_DH        14.241245
AVG_DH          14.941634
MINUTES_DH      15.297387
EXPERIENCIA     15.308505
DIAS_EMP         0.000000
dtype: float64

In [ ]:
df_training_dataset['ESTUDIO_PREV'].value_counts()

ENGINEERING    3850
COMMERCIAL     2077
DEVELOPMENT    1427
BUSINESS        203
MARKETING       108
Name: ESTUDIO_PREV, dtype: int64

In [ ]:
df_training_dataset['TRACK_DH'].value_counts()

DATA            3464
PROGRAMACION    3033
EJECUTIVO        698
MARKETING        519
Name: TRACK_DH, dtype: int64

In [ ]:
df_training_dataset['ESTUDIO_PREV'] = df_training_dataset['ESTUDIO_PREV'].replace({"MARKETING":"ESTUDIO_MARKETING"})
df_training_dataset['TRACK_DH'] = df_training_dataset['TRACK_DH'].replace({"MARKETING":"TRACK_MARKETING"})

In [ ]:
#@title tipo de variables
types = pd.DataFrame(df_training_dataset.dtypes)
print(types.groupby(0).size())

0
int64      1
float64    5
object     5
dtype: int64


In [ ]:
#@title Evaluando variables categoricas

categoricas = types.index[types[0]=='O'].values
for line in categoricas:
 print("La variable "+ line +" contiene:",str(len(df_training_dataset[line].unique()))+" distintos valores")

La variable GENERO contiene: 3 distintos valores
La variable RESIDENCIA contiene: 4 distintos valores
La variable NV_ESTUDIO contiene: 4 distintos valores
La variable ESTUDIO_PREV contiene: 6 distintos valores
La variable TRACK_DH contiene: 5 distintos valores


In [ ]:
#@title Creando variables dummies
df_dummies = df_training_dataset.copy()
for col in categoricas:
  df_dummies = pd.concat([df_dummies,(pd.get_dummies(df_dummies[col])).astype(int)], axis=1)
  df_dummies.drop(columns=[col],inplace=True)

df_dummies.drop(["Unnamed: 0"],axis=1, inplace=True)
print(df_dummies.shape)
df_dummies.tail(5)

(8995, 22)


,EDAD,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP,FEMENINO,MASCULINO,ARGENTINA,BRAZIL,MEXICO,...,UNIVERSITARY,BUSINESS,COMMERCIAL,DEVELOPMENT,ENGINEERING,ESTUDIO_MARKETING,DATA,EJECUTIVO,PROGRAMACION,TRACK_MARKETING
8990,29.0,4.0,4701.6,9.1,86.0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
8991,34.0,3.4,4646.2,16.8,95.0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
8992,28.0,NaN,3315.1,5.6,95.0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
8993,23.0,3.3,4437.8,0.9,87.0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
8994,36.0,3.4,4600.8,19.6,88.0,0,1,1,0,0,...,1,0,1,0,0,0,1,0,0,0


Creando Label Encoder

In [ ]:
df_training_dataset.tail(5)

,Unnamed: 0,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
8990,9995,29.0,MASCULINO,ARGENTINA,TERTIARY,DEVELOPMENT,NaN,4.0,4701.6,9.1,86.0
8991,9996,34.0,NaN,ARGENTINA,UNIVERSITARY,ENGINEERING,PROGRAMACION,3.4,4646.2,16.8,95.0
8992,9997,28.0,FEMENINO,ARGENTINA,POST_GRADUATE,ENGINEERING,EJECUTIVO,NaN,3315.1,5.6,95.0
8993,9998,23.0,MASCULINO,MEXICO,TERTIARY,ENGINEERING,PROGRAMACION,3.3,4437.8,0.9,87.0
8994,9999,36.0,MASCULINO,ARGENTINA,UNIVERSITARY,COMMERCIAL,DATA,3.4,4600.8,19.6,88.0


In [ ]:
from sklearn.preprocessing import LabelEncoder

df_encoder = df_training_dataset.copy()
# Instantiate LabelEncoder
le = LabelEncoder()
df_encoder_columns = ["GENERO","RESIDENCIA","NV_ESTUDIO","ESTUDIO_PREV","TRACK_DH"]
# Encode single column status
df_encoder[df_encoder_columns] = df_encoder[df_encoder_columns].apply(LabelEncoder().fit_transform)

In [ ]:
df_encoder.head()

,Unnamed: 0,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
0,1000,37.0,0,1,2,5,2,3.1,4131.5,21.3,97.0
1,1001,40.0,0,3,2,1,2,3.1,4160.4,25.2,102.0
2,1002,35.0,2,0,2,2,0,3.1,4087.6,18.0,89.0
3,1003,33.0,0,1,2,3,4,3.1,4043.2,13.6,96.0
4,1004,29.0,2,1,2,2,4,3.6,4688.0,NaN,83.0


Imputar datos

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np


impute_zeros = SimpleImputer(
    missing_values=np.nan,
    strategy='most_frequent',
    fill_value=0,
    verbose=0,
    copy=True
)

In [ ]:
print("Valores nulos del df_training_dataset antes de la transformación SimpleImputer: \n\n{}\n".format(df_encoder.isnull().sum(axis = 0)))
# Aplicando la transformación ``SimpleImputer`` en conjunto de datos base
impute_zeros.fit(X=df_encoder)

# Reconstruyendo un Pandas DataFrame con los resultados
df = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_encoder
    ),
    columns=df_encoder.columns
)

# Mostrando los datos nulos del dataset despues de la segunda transformación (df)
print("Valores nulos del df_training_dataset despues de la transformación SimpleImputer: \n\n{}\n".format(df.isnull().sum(axis = 0)))

Valores nulos del df_training_dataset antes de la transformación SimpleImputer: 

Unnamed: 0         0
EDAD            1327
GENERO             0
RESIDENCIA         0
NV_ESTUDIO         0
ESTUDIO_PREV       0
TRACK_DH           0
AVG_DH          1344
MINUTES_DH      1376
EXPERIENCIA     1377
DIAS_EMP           0
dtype: int64

Valores nulos del df_training_dataset despues de la transformación SimpleImputer: 

Unnamed: 0      0
EDAD            0
GENERO          0
RESIDENCIA      0
NV_ESTUDIO      0
ESTUDIO_PREV    0
TRACK_DH        0
AVG_DH          0
MINUTES_DH      0
EXPERIENCIA     0
DIAS_EMP        0
dtype: int64



In [ ]:
types = pd.DataFrame(df.dtypes)
print("Tipos de variables", types.groupby(0).size())

Tipos de variables 0
float64    11
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8995 entries, 0 to 8994
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8995 non-null   float64
 1   EDAD          8995 non-null   float64
 2   GENERO        8995 non-null   float64
 3   RESIDENCIA    8995 non-null   float64
 4   NV_ESTUDIO    8995 non-null   float64
 5   ESTUDIO_PREV  8995 non-null   float64
 6   TRACK_DH      8995 non-null   float64
 7   AVG_DH        8995 non-null   float64
 8   MINUTES_DH    8995 non-null   float64
 9   EXPERIENCIA   8995 non-null   float64
 10  DIAS_EMP      8995 non-null   float64
dtypes: float64(11)
memory usage: 773.1 KB


In [ ]:
df

,Unnamed: 0,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
0,1000.0,37.0,0.0,1.0,2.0,5.0,2.0,3.1,4131.5,21.3,97.0
1,1001.0,40.0,0.0,3.0,2.0,1.0,2.0,3.1,4160.4,25.2,102.0
2,1002.0,35.0,2.0,0.0,2.0,2.0,0.0,3.1,4087.6,18.0,89.0
3,1003.0,33.0,0.0,1.0,2.0,3.0,4.0,3.1,4043.2,13.6,96.0
4,1004.0,29.0,2.0,1.0,2.0,2.0,4.0,3.6,4688.0,16.1,83.0
...,...,...,...,...,...,...,...,...,...,...,...
8990,9995.0,29.0,1.0,0.0,1.0,2.0,4.0,4.0,4701.6,9.1,86.0
8991,9996.0,34.0,2.0,0.0,2.0,3.0,2.0,3.4,4646.2,16.8,95.0
8992,9997.0,28.0,0.0,0.0,0.0,3.0,1.0,3.6,3315.1,5.6,95.0
8993,9998.0,23.0,1.0,2.0,1.0,3.0,2.0,3.3,4437.8,0.9,87.0


Para nuestro modelo predictivo. ¿ Cuales columnas son realemnte importantes ?

In [ ]:
df_final = df.copy() 
df_final.drop(columns=["Unnamed: 0","EDAD","GENERO"], inplace=True)
df_final

,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
0,1.0,2.0,5.0,2.0,3.1,4131.5,21.3,97.0
1,3.0,2.0,1.0,2.0,3.1,4160.4,25.2,102.0
2,0.0,2.0,2.0,0.0,3.1,4087.6,18.0,89.0
3,1.0,2.0,3.0,4.0,3.1,4043.2,13.6,96.0
4,1.0,2.0,2.0,4.0,3.6,4688.0,16.1,83.0
...,...,...,...,...,...,...,...,...
8990,0.0,1.0,2.0,4.0,4.0,4701.6,9.1,86.0
8991,0.0,2.0,3.0,2.0,3.4,4646.2,16.8,95.0
8992,0.0,0.0,3.0,1.0,3.6,3315.1,5.6,95.0
8993,2.0,1.0,3.0,2.0,3.3,4437.8,0.9,87.0
